In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath('..')))

In [ ]:
# portfolio_optimization.ipynb
import os
import sys
import logging
from src.portfolio_optimizer import PortfolioOptimizer

# Add the src directory to the Python path
sys.path.append(os.path.join(os.path.abspath('..'), 'src'))

In [ ]:
# File paths for the forecast data
tsla_forecast_path = '../models/best_models/forecasts_TSLA.csv'
bnd_forecast_path = '../models/best_models/forecasts_BND.csv'
spy_forecast_path = '../models/best_models/forecasts_SPY.csv'

# Create an instance of the PortfolioOptimizer
optimizer = PortfolioOptimizer(tsla_csv=tsla_forecast_path, 
                               bnd_csv=bnd_forecast_path, 
                               spy_csv=spy_forecast_path, 
                               logger=logger)

# Generate a summary of the portfolio
summary = optimizer.summary()

# Visualize the portfolio performance
optimizer.visualize_portfolio_performance()